In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from functools import partial

img1 = plt.imread("/Users/tristanbrigham/GithubProjects/AzimuthInternship/DrLuckingProjects/STMProject1/firstSTM.png")
img2 = plt.imread("/Users/tristanbrigham/GithubProjects/AzimuthInternship/DrLuckingProjects/STMProject1/image2.png")
img3 = plt.imread("/Users/tristanbrigham/GithubProjects/AzimuthInternship/DrLuckingProjects/STMProject1/image3.png")

img2 = cv.resize(img2, dsize=(img1.shape[1], img1.shape[0]), interpolation=cv.INTER_NEAREST)
img3 = cv.resize(img3, dsize=(img1.shape[1], img1.shape[0]), interpolation=cv.INTER_NEAREST)

print("Size1: {}".format(img1.shape))
print("Size2: {}".format(img2.shape))
print("Size3: {}".format(img3.shape))

finalArray = np.zeros((436, 17, 15, 4))
# finalArray2 = np.zeros((336, 17, 15, 4))
# finalArray3 = np.zeros((336, 17, 15, 4))

img2Array = np.zeros((336, 17, 15, 4))
img3Array = np.zeros((336, 17, 15, 4))

yAmt = int(img1.shape[1] /  14)
offsetY = int(yAmt / 2)

xAmt = int(img1.shape[0] / 12)
# offsetX = int(xAmt / 2)
offsetX = xAmt     #uncomment the above line and comment this one to add overlap

count = 0
x = xAmt

possibleResults = [0, 1]

while x < img1.shape[1]:
    y = yAmt
    while y < img1.shape[0] - offsetY:
        imgTemp1 = img1[y - yAmt:y, x - xAmt:x]
        finalArray[count] = imgTemp1
        
        imgTemp2 = img2[y - yAmt:y, x - xAmt:x]
        img2Array[count] = imgTemp2

        imgTemp3 = img3[y - yAmt:y, x - xAmt:x]
        img3Array[count] = imgTemp3
        count += 1
        y += offsetY
        if(count > 1000):
            exit()
    x += offsetX
# 
# finalArray[336: 672] = finalArray2
# finalArray[672: 1008] = finalArray3

# count = 0
# for img in finalArray[:1008]:
#     cv.imwrite("Img{}.png".format(count), img)
#     count += 1
    
print(finalArray.shape)

imagesWithDefectsNumbers = [23, 24, 25, 179, 199, 200, 201, 221, 222, 223]
imagesWithDefects = np.zeros((10, 17, 15, 4))

# labels = np.zeros((1108)) #going to be a zero if there is no defect
# labels[1008 : 1109] = np.ones((100))

labels = np.zeros((436))
labels[336: 436] = np.ones((100))

count = 0
for i in imagesWithDefectsNumbers:
    labels[i] = 1
    imagesWithDefects[count] = finalArray[i]
    count += 1

for i in range(10):
    # finalArray[1008 + (i * 10) : 1018 + (i * 10)] = imagesWithDefects[0 : 10]
    finalArray[336 + (i * 10) : 346 + (i * 10)] = imagesWithDefects[0 : 10]

xSections = int(img1.shape[1]/finalArray.shape[2])
ySections = int(336/xSections) #using 336 because count is wonky -- should be final Array y shape

print("x sect # is {} and y sect # is {}".format(xSections, ySections))
# 21 sections in y direction
# 16 sections in x direction


Size1: (187, 248, 4)
Size2: (187, 248, 4)
Size3: (187, 248, 4)
(436, 17, 15, 4)
x sect # is 16 and y sect # is 21


In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")
model = keras.models.Sequential([
    #first layer is 64 7x7 filters
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[17, 15, 4]),
    # pool reduce each spatial dimension by factor of 2
    keras.layers.MaxPooling2D(pool_size=2),
    #Repeat 2 convolution layers followed by one pool twice
    #number of filters increases as go further along
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    #need to flatten input for dense layer because need 1D array for input
    keras.layers.Flatten(),
    #Fully connected network, dropout layer to reduce overfit
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=2, activation='softmax'),
])


model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(finalArray, labels, epochs=35, validation_data=(finalArray[300:, :, :, :], labels[300:]))
score = model.evaluate(finalArray[325:, :, :, :], labels[325:])

Epoch 1/35
14/14 [==============================] - 5s 161ms/step - loss: 0.6255 - accuracy: 0.7339 - val_loss: 1.1451 - val_accuracy: 0.2647
Epoch 2/35
14/14 [==============================] - 2s 126ms/step - loss: 0.6426 - accuracy: 0.7454 - val_loss: 0.8294 - val_accuracy: 0.2647
Epoch 3/35
14/14 [==============================] - 2s 106ms/step - loss: 0.6256 - accuracy: 0.7729 - val_loss: 0.9189 - val_accuracy: 0.3382
Epoch 4/35
14/14 [==============================] - 2s 141ms/step - loss: 0.3917 - accuracy: 0.8303 - val_loss: 0.9559 - val_accuracy: 0.5588
Epoch 5/35
 9/14 [==================>...........] - ETA: 0s - loss: 0.3388 - accuracy: 0.8299

In [ ]:
# print("Accuracy: {}".format(score[0]['accuracy']))
X_new = finalArray # pretend we have new images
y_pred = model.predict(X_new)
for i in range(436):
    pred = y_pred[i]
    if possibleResults[pred.argmax()] != labels[i]:
        print("Validation Image {} was incorrect :: {}".format(i, labels[count]))

newPred2 = model.predict(img2Array)
count = -1
defective2 = []
nonDefective2 = []
for pred in newPred2:
    count += 1
    if possibleResults[pred.argmax()] == 1:
        defective2.append(count)
        x = 15 * int(count / ySections) + xAmt
        y = 17 * (count % ySections) + yAmt
        cv.rectangle(img2, (x - xAmt, y - yAmt), (x, y), (0, 255, 0))
    else:
        nonDefective2.append(count)

print("2:\nDEFECTIVE: {} \n\nNONDEFECTIVE: {}\n\n".format(defective2, nonDefective2))

In [ ]:
newPred3 = model.predict(img3Array)
count = 0
defective3 = []
nonDefective3 = []
for pred in newPred3:
    if possibleResults[pred.argmax()] == 1:
        defective3.append(count) 
        x = 15 * int(count / ySections) + xAmt
        y = 17 * (count % ySections) + yAmt
        cv.rectangle(img3, (x - xAmt, y - yAmt), (x, y), (0, 255, 0))
    else:
        nonDefective3.append(count)
    count += 1

print("3:\nDEFECTIVE: {} \n\nNONDEFECTIVE: {}\n\n".format(defective3, nonDefective3))

In [ ]:
factor = 4

img2 = cv.resize(img2, dsize=(img1.shape[1]*factor, img1.shape[0]*factor), interpolation=cv.INTER_NEAREST)
img3 = cv.resize(img3, dsize=(img1.shape[1]*factor, img1.shape[0]*factor), interpolation=cv.INTER_NEAREST)

cv.imshow("Img2 Bounded", img2)
cv.imshow("Img3 Bounded", img3)
if cv.waitKey(0) & 0xFF == ord('q'):
    cv.destroyAllWindows()